In [2]:
from datetime import datetime
import random
import os
import pygame
import pymunk
import pymunk.pygame_util
import numpy as np
import cv2

In [27]:
'''
VIDEO_FILENAME = f"type4_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"

# Global variables
SIMULATION_TIME =  11 
FINAL_VIDEO_TIME = SIMULATION_TIME -1

FPS = 100
TOTAL_BALLS = 12
BALL_RADIUS = 30
WIDTH, HEIGHT = 512, 512
COLORS = {'#CB4154': (203, 65, 84), '#5CB3FF': (92, 179, 255), '#5A8F00': (90, 143, 0), '#FFF44F': (255, 244, 79)}
MASSES = {'#CB4154': 1, '#5CB3FF': 2, '#5A8F00': 4, '#FFF44F': 8}
BRICK_SIZE = 32
'''

'\nVIDEO_FILENAME = f"type4_data_{datetime.now().strftime(\'%Y%m%d_%H%M%S\')}.mp4"\n\n# Global variables\nSIMULATION_TIME =  11 \nFINAL_VIDEO_TIME = SIMULATION_TIME -1\n\nFPS = 100\nTOTAL_BALLS = 12\nBALL_RADIUS = 30\nWIDTH, HEIGHT = 512, 512\nCOLORS = {\'#CB4154\': (203, 65, 84), \'#5CB3FF\': (92, 179, 255), \'#5A8F00\': (90, 143, 0), \'#FFF44F\': (255, 244, 79)}\nMASSES = {\'#CB4154\': 1, \'#5CB3FF\': 2, \'#5A8F00\': 4, \'#FFF44F\': 8}\nBRICK_SIZE = 32\n'

In [28]:
# Global Variables

VIDEO_FILENAME = f"type2_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"


SIMULATION_TIME =  11 
FINAL_VIDEO_TIME = SIMULATION_TIME -1

FPS = 100
TOTAL_BALLS = 12
BALL_RADIUS = 30
WIDTH, HEIGHT = 512, 512
COLORS = {
    'red': '#FF0000',
    'green': '#00FF00',
    'yellow': '#FFFF00',
    'blue': '#0000FF'
}

MASSES = {'red': 1, 'yellow': 2, 'green': 4, 'blue': 8}
BRICK_SIZE = 32

In [29]:
def create_space_with_random_bricks():
    """Initialize the Pymunk space, create walls, and add random bricks inside the free space."""
    space = pymunk.Space()
    space.gravity = (0, 0)  # No gravity

    # Create grid-like wall frames
    walls = []
    grid_color = (169, 169, 169)  # Gray for grid bricks
    for i in range(0, WIDTH, BRICK_SIZE):
        # Top and bottom walls
        walls.append(pymunk.Poly(space.static_body, [
            (i, 0), (i + BRICK_SIZE, 0), (i + BRICK_SIZE, BRICK_SIZE), (i, BRICK_SIZE)
        ]))
        walls.append(pymunk.Poly(space.static_body, [
            (i, HEIGHT - BRICK_SIZE), (i + BRICK_SIZE, HEIGHT - BRICK_SIZE),
            (i + BRICK_SIZE, HEIGHT), (i, HEIGHT)
        ]))
    for j in range(0, HEIGHT, BRICK_SIZE):
        # Left and right walls
        walls.append(pymunk.Poly(space.static_body, [
            (0, j), (BRICK_SIZE, j), (BRICK_SIZE, j + BRICK_SIZE), (0, j + BRICK_SIZE)
        ]))
        walls.append(pymunk.Poly(space.static_body, [
            (WIDTH - BRICK_SIZE, j), (WIDTH, j), (WIDTH, j + BRICK_SIZE), (WIDTH - BRICK_SIZE, j + BRICK_SIZE)
        ]))

    # Add random internal bricks
    occupied_positions = set()
    internal_bricks = []
    for _ in range(10):  # Place 10 random bricks
        while True:
            # Generate random grid positions within the free space
            x = random.randint(1, (WIDTH // BRICK_SIZE) - 2) * BRICK_SIZE
            y = random.randint(1, (HEIGHT // BRICK_SIZE) - 2) * BRICK_SIZE
            if (x, y) not in occupied_positions:  # Ensure no overlapping positions
                occupied_positions.add((x, y))
                brick = pymunk.Poly(space.static_body, [
                    (x, y), (x + BRICK_SIZE, y), (x + BRICK_SIZE, y + BRICK_SIZE), (x, y + BRICK_SIZE)
                ])
                brick.elasticity = 1.0  # Completely elastic collisions
                internal_bricks.append(brick)
                break

    # Add walls and bricks to the space
    for wall in walls:
        wall.elasticity = 1.0  # Completely elastic collisions
        space.add(wall)
    space.add(*internal_bricks)

    return space, walls, internal_bricks, grid_color


In [30]:
def create_balls(space):
    """Create balls with random positions, velocities, and colors."""
    import random

    balls = []
    for _ in range(TOTAL_BALLS):
        color = random.choice(list(COLORS.keys()))
        mass = MASSES[color]
        moment = pymunk.moment_for_circle(mass, 0, BALL_RADIUS)
        body = pymunk.Body(mass, moment)
        body.position = random.uniform(BALL_RADIUS, WIDTH - BALL_RADIUS), random.uniform(BALL_RADIUS, HEIGHT - BALL_RADIUS)
        body.velocity = random.uniform(-150, 150), random.uniform(-150, 150)  # Random initial velocity
        shape = pymunk.Circle(body, BALL_RADIUS)
        shape.elasticity = 1.0  # Elastic collisions
        shape.friction = 0  # No friction
        shape.color = COLORS[color]
        space.add(body, shape)
        balls.append(shape)
    return balls


In [31]:
def simulate_and_record_with_internal_bricks(space, walls, internal_bricks, grid_color, balls):
    """Simulate the environment, constrain balls, and record high-resolution frames with random bricks."""
    pygame.init()
    screen = pygame.Surface((WIDTH, HEIGHT))
    clock = pygame.time.Clock()
    draw_options = pymunk.pygame_util.DrawOptions(screen)

    # Setup OpenCV video writer
    high_res_width = WIDTH * 2
    high_res_height = HEIGHT * 2
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(VIDEO_FILENAME, fourcc, FPS, (high_res_width, high_res_height))

    # Simulation loop
    steps = int(SIMULATION_TIME * FPS)
    start_frame = int(FPS)  # Skip the first second
    for frame in range(steps):
        # Clear the screen
        screen.fill((255, 255, 255))  # White background

        # Draw grid-like walls
        for wall in walls:
            pygame.draw.polygon(screen, grid_color, [(int(p[0]), int(p[1])) for p in wall.get_vertices()])
            pygame.draw.polygon(screen, (0, 0, 0), [(int(p[0]), int(p[1])) for p in wall.get_vertices()], width=1)

        # Draw random internal bricks
        for brick in internal_bricks:
            pygame.draw.polygon(screen, grid_color, [(int(p[0]), int(p[1])) for p in brick.get_vertices()])
            pygame.draw.polygon(screen, (0, 0, 0), [(int(p[0]), int(p[1])) for p in brick.get_vertices()], width=1)

        # Constrain balls within the wall frame and draw them
        for ball in balls:
            # Draw the balls
            pygame.draw.circle(
                screen, ball.color,
                (int(ball.body.position.x), int(ball.body.position.y)),
                BALL_RADIUS
            )
            pygame.draw.circle(
                screen, (0, 0, 0),
                (int(ball.body.position.x), int(ball.body.position.y)),
                BALL_RADIUS, width=1
            )

        # Skip the first second by not writing those frames
        if frame >= start_frame:
            # Convert Pygame surface to OpenCV frame
            frame_data = pygame.surfarray.array3d(screen)
            frame_data = np.transpose(frame_data, (1, 0, 2))  # Convert to OpenCV format
            high_res_frame = cv2.resize(frame_data, (high_res_width, high_res_height), interpolation=cv2.INTER_LINEAR)
            video.write(cv2.cvtColor(high_res_frame, cv2.COLOR_RGB2BGR))

        # Step the simulation
        space.step(1 / FPS)
        clock.tick(FPS)

    video.release()
    pygame.quit()
    print(f"Simulation complete. Video saved as {VIDEO_FILENAME}.")


In [32]:
if __name__ == "__main__":

    for _ in range(1):        
        
        VIDEO_FILENAME = f"type4_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
        simulation_space, simulation_walls, internal_bricks, grid_wall_color = create_space_with_random_bricks()
        balls = create_balls(simulation_space)
        simulate_and_record_with_internal_bricks(simulation_space, simulation_walls, internal_bricks, grid_wall_color, balls)


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1342:(snd_func_refer) error evaluating name
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5727:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM default


Simulation complete. Video saved as type4_data_20250105_032949.mp4.


In [ ]:
############################################################## MAGIC CODE ################################################################
# ffmpeg -i ./dataset/type4_data_20250102_071333.mp4 -vcodec libx264 -acodec aac ./dataset/converted_video.mp4

In [ ]:
import os
import subprocess

# Directories
input_dir = "/home/gridsan/yyao/Research_Projects/Particle_Simulation/graph_video_prediction/dataset/data_gen/raw/B"
output_dir = "/home/gridsan/yyao/Research_Projects/Particle_Simulation/graph_video_prediction/dataset/data_gen/converted/B"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Loop through all files in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith(".mp4"):
        input_path = os.path.join(input_dir, file_name)
        
        # Add "_converted" before the file extension
        base_name, ext = os.path.splitext(file_name)
        output_file_name = f"{base_name}_converted{ext}"
        output_path = os.path.join(output_dir, output_file_name)
        
        # ffmpeg command to convert the video
        command = [
            "ffmpeg", "-i", input_path, 
            "-vcodec", "libx264", 
            "-acodec", "aac", 
            output_path
        ]
        
        # Run the command
        print(f"Converting {file_name} to {output_file_name}...")
        subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

print(f"All files have been converted and saved in the {output_dir}")


Converting type_B_20250107_052721.mp4 to type_B_20250107_052721_converted.mp4...
Converting type_B_20250106_223511.mp4 to type_B_20250106_223511_converted.mp4...
Converting type_B_20250107_052747.mp4 to type_B_20250107_052747_converted.mp4...
Converting type_B_20250107_025050.mp4 to type_B_20250107_025050_converted.mp4...
Converting type_B_20250107_012333.mp4 to type_B_20250107_012333_converted.mp4...
Converting type_B_20250107_053211.mp4 to type_B_20250107_053211_converted.mp4...
Converting type_B_20250107_024516.mp4 to type_B_20250107_024516_converted.mp4...
Converting type_B_20250106_235412.mp4 to type_B_20250106_235412_converted.mp4...
Converting type_B_20250106_231450.mp4 to type_B_20250106_231450_converted.mp4...
Converting type_B_20250107_080335.mp4 to type_B_20250107_080335_converted.mp4...
Converting type_B_20250107_075337.mp4 to type_B_20250107_075337_converted.mp4...
Converting type_B_20250107_045924.mp4 to type_B_20250107_045924_converted.mp4...
Converting type_B_20250106_2